In [ ]:
# Upgrade Oracle ADS to pick up latest features and maintain compatibility with Oracle Cloud Infrastructure.
!pip install --pre --no-deps oracle-ads==2.9.0rc0

Oracle Data Science service sample notebook.

Copyright (c) 2022 Oracle, Inc. All rights reserved. Licensed under the [Universal Permissive License v 1.0](https://oss.oracle.com/licenses/upl).

***

# <font color="red">Feature store quickstart</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color="teal">Oracle Cloud Infrastructure Data Science Service.</font></p>

---
# Overview:
---
Managing many datasets, data-sources and transformations for machine learning is complex and costly. Poorly cleaned data, data issues, bugs in transformations, data drift and training serving skew all leads to increased model development time and worse model performance. Here, feature store is well positioned to solve many of the problems since it provides a centralised way to transform and access data for training and serving time and helps defines a standardised pipeline for ingestion of data and querying of data.

Compatible conda pack: [PySpark 3.2 and Feature Store](https://docs.oracle.com/iaas/data-science/using/conda-pyspark-fam.htm) for CPU on Python 3.8 (version 1.0)

## Contents:

- <a href="#concepts">1. Introduction</a>
- <a href='#pre-requisites'>2. Pre-requisites</a>
    - <a href='#setup'>2.1 Setup</a>
    - <a href='#policies'>2.2 Policies</a>
    - <a href='#prerequisites_authentication'>2.3 Authentication</a>
    - <a href='#prerequisites_variables'>2.4 Variables</a>
- <a href='#featurestore_overview'>3. Feature store quickstart using APIs</a>
    - <a href='#explore_data'>3.1 Exploration of data</a>
    - <a href='#featurestore_entities'>3.2 Create feature store logical entities</a>
        - <a href='#create_feature_store'>3.2.1 Create feature store</a>
        - <a href='#create_entity'>3.2.2 Create business entity in feature store</a>
        - <a href='#create_transformation'>3.2.3 Create transformation in feature store</a>
        - <a href='#create_featuregroup'>3.2.4 Create feature group and upload data to feature group</a>
    - <a href='#explore_featuregroup'>3.3 Explore feature group</a>
    - <a href='#create_dataset'>3.4 Create dataset from multiple or one feature group</a>
    - <a href='#explore_dataset'>3.3 Explore dataset</a>
    - <a href='#featurestore_yaml'>4. Feature store quickstart using YAML</a>
    - <a href='#ref'>5. References</a>

---

**Important:**

Placeholder text for required values are surrounded by angle brackets that must be removed when adding the indicated content. For example, when adding a database name to `database_name = "<database_name>"` would become `database_name = "production"`.

---

Datasets are provided as a convenience.  Datasets are considered third-party content and are not considered materials under your agreement with Oracle.

`Citibike` dataset is used in this notebook.You can access the citibike dataset license [here](https://ride.citibikenyc.com/data-sharing-policy)

---

<a id="concepts"></a>
# 1. Introduction

Oracle feature store is a stack based solution that is deployed in the customer enclave using OCI resource manager. Customer can stand up the service with infrastructure in their own tenancy. The service consists of API which are deployed in customer tenancy using resource manager.

The following are some key terms that will help you understand OCI Data Science Feature Store:


* **Feature Vector**: Set of feature values for any one primary/identifier key. Eg. All/subset of features of customer id ‘2536’ can be called as one feature vector.

* **Feature**: A feature is an individual measurable property or characteristic of a phenomenon being observed.

* **Entity**: An entity is a group of semantically related features. The first step a consumer of features would typically do when accessing the feature store service is to list the entities and the entities associated features. Another way to look at it is that an entity is an object or concept that is described by its features. Examples of entities could be customer, product, transaction, review, image, document, etc.

* **Feature Group**: A feature group in a feature store is a collection of related features that are often used together in ml models. It serves as an organizational unit within the feature store for users to manage, version and share features across different ml projects. By organizing features into groups, data scientists and ml engineers can efficiently discover, reuse and collaborate on features reducing the redundant work and ensuring consistency in feature engineering.

* **Feature Group Job**: Feature group job is the execution instance of a feature group. Each feature group job will include validation results and statistics results.

* **Dataset**: A dataset is a collection of feature that are used together to either train a model or perform model inference.

* **Dataset Job**: Dataset job is the execution instance of a dataset. Each dataset job will include validation results and statistics results.

<a id='pre-requisites'></a>
# 2. Pre-requisites to Running this Notebook 

Notebook Sessions are accessible through the following conda environment: 

* **PySpark 3.2 and Feature Store Python 3.8 (fspyspark32_p38_cpu_v1)**

You can customize `fspyspark32_p38_cpu_v1`, publish it, and use it as a runtime environment for a Notebook session cluster. 

<a id='setup'></a>
### 2.1. Setup

To set up the environment, a `spark-defaults.conf` must be configured. Data Catalog Metastore id must also be provided.

<a id='setup_spark-defaults'></a>
### `spark-defaults.conf`

The `spark-defaults.conf` file is used to define the properties that are used by Spark. A templated version is installed when you install a Data Science conda environment that supports PySpark. However, you must update the template so that the Data Catalog metastore can be accessed. You can do this manually. However, the `odsc data-catalog config` commandline tool is ideal for setting up the file because it gathers information about your environment, and uses that to build the file.

The `odsc data-catalog config` command line tool needs the `--metastore` option to define the Data Catalog metastore OCID. No other command line option is needed because settings have default values, or they take values from your notebook session environment. Following are common parameters that you may need to override.

The `--authentication` option sets the authentication mode. It supports resource principal and API keys. The preferred method for authentication is resource principal, which is sent with `--authentication resource_principal`. If you want to use API keys, then use the `--authentication api_key` option. If the `--authentication` isn't specified, API keys are used. When API keys are used, information from the OCI configuration file is used to create the `spark-defaults.conf` file.

Object Storage and Data Catalog are regional services. By default, the region is set to the region your notebook session is running in. This information is taken from the environment variable, `NB_REGION`. Use the `--region` option to override this behavior.

The default location of the `spark-defaults.conf` file is `/home/datascience/spark_conf_dir` as defined in the `SPARK_CONF_DIR` environment variable. Use the `--output` option to define the directory where to write the file.

You need to determine what settings are appropriate for your configuration. However, the following works for most configurations and is run in a terminal window.

```bash
odsc data-catalog config --authentication resource_principal --metastore <metastore_id>
```
For more assistance, use the following command in a terminal window:

```bash
odsc data-catalog config --help
```

<a id='policies'></a>
### 2.2. Policies
This section covers the creation of dynamic groups and policies needed to use the service.

* [About Data Science Policies](https://docs.oracle.com/iaas/data-science/using/policies.htm)
* [Data Catalog Metastore Required Policies](https://docs.oracle.com/en-us/iaas/data-catalog/using/metastore.htm)

<a id="prerequisites_authentication"></a>
### 2.3. Authentication
The [Oracle Accelerated Data Science SDK (ADS)](https://docs.oracle.com/iaas/tools/ads-sdk/latest/index.html) controls the authentication mechanism with the notebook Spark cluster.<br> 
To setup authentication use the ```ads.set_auth("resource_principal")``` or ```ads.set_auth("api_key")```. 

In [ ]:
import ads
ads.set_auth(auth="resource_principal", client_kwargs={"fs_service_endpoint": "http://{api_gateway}/20230101"})

<a id="prerequisites_variables"></a>
### 2.4. Variables
To run this notebook, you must provide some information about your tenancy configuration. To create and run a feature store, you must specify a `<compartment_id>` and `<metastore_id>` which is the OCID of the Data Catalog metastore. The [Data Catalog Hive Metastore](https://docs.oracle.com/en-us/iaas/data-catalog/using/metastore.htm) provides schema definitions for objects in structured and unstructured data assets. The Metastore is the central metadata repository to understand tables backed by files on object storage and the metastore id of hive metastore is tied to feature store construct of feature store service.

In [ ]:
import os

compartment_id = os.environ.get("NB_SESSION_COMPARTMENT_OCID")
metastore_id = "<metastore_id>"

<a id="featurestore_overview"></a>
# 3. Feature store quick start using APIs
By default the **PySpark 3.2 and Feature Store Python 3.8** conda environment includes pre-installed [great-expectations](https://legacy.docs.greatexpectations.io/en/latest/reference/core_concepts/validation.html) library. In an ADS feature store module, you can either use the Python programmatic or YAML interface to define feature store entities. Below section describes how to create feature store entities using programmatic interface.

In [ ]:
import pandas as pd 
from ads.feature_store.feature_store import FeatureStore
from ads.feature_store.dataset import Dataset
from ads.feature_store.feature_group import FeatureGroup
from ads.feature_store.feature_store_registrar import FeatureStoreRegistrar
from ads.feature_store.common.enums import ExpectationType
from great_expectations.core import ExpectationSuite, ExpectationConfiguration
from ads.feature_store.transformation import TransformationMode

<a id="explore_data"></a>
### 3.1 Exploration of data

In [ ]:
bike_df = pd.read_csv("data/201901-citibike-tripdata.csv")

In [ ]:
bike_df.head()

In [ ]:
bike_df.columns = bike_df.columns.str.replace(' ', '')

<a id="featurestore_entities"></a>
### 3.2. Create feature store logical entities

<a id="create_feature_store"></a>
#### 3.2.1 Feature Store

Feature store is the top level entity for feature store service.
Call the ```.create()``` method of the Feature store instance to create a feature store.

In [ ]:
feature_store_resource = (
    FeatureStore().
    with_description("Data consisting of bike riders data").
    with_compartment_id(compartment_id).
    with_display_name("Bike rides").
    with_offline_config(metastore_id=metastore_id)
)

In [ ]:
feature_store = feature_store_resource.create()

<a id="create_entity"></a>
#### 3.2.2 Entity
An entity is a group of semantically related features. The first step a consumer of features would typically do when accessing the feature store service is to list the entities and the entities associated features. Another way to look at it is that an entity is an object or concept that is described by its features. Examples of entities could be customer, product, transaction, review, image, document, etc.

In [ ]:
entity = feature_store.create_entity(
    display_name="Bike rides",
    description="description for bike riders"
)

<a id="create_transformation"></a>
#### 3.2.3 Transformation
Transformations in a feature store refers to the operations and processes applied to raw data to create, modify or derive new features that can be used as inputs for ML Models

In [ ]:
def is_round_trip(bike_df):
    bike_df['roundtrip'] = bike_df['startstationid'] == bike_df['endstationid']
    return bike_df

In [ ]:
transformation = feature_store.create_transformation(
    transformation_mode=TransformationMode.PANDAS,
    source_code_func=is_round_trip,
    display_name="is_round_trip",
)
transformation

<a id="create_featuregroup"></a>
#### 3.2.4 Feature group
A feature group is an object that represents a logical group of time-series feature data as it is found in a datasource. 

<a id="=featuregroup_validation"></a>
##### 3.2.4.1 Associate Expectation Suite
Feature validation is the process of checking the quality and accuracy of the features used in a machine learning model.Feature store allows you to define expectation on the data which is being materialized into feature group and dataset.This is achieved using open source library Great Expectations.

An Expectation is a verifiable assertion about your data. You can define expectation as below:

In [ ]:
expectation_suite = ExpectationSuite(expectation_suite_name="feature_definition")
expectation_suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={"column": "stoptime"}
    )
)

In [ ]:
feature_group_bike = (
    FeatureGroup()
    .with_feature_store_id(feature_store.id)
    .with_primary_keys(["bikeid"])
    .with_name("bike_feature_group")
    .with_entity_id(entity.id)
    .with_compartment_id(compartment_id)
    .with_schema_details_from_dataframe(bike_df)
    .with_expectation_suite(expectation_suite, ExpectationType.LENIENT)
    .with_transformation_id(transformation.id)
)

In [ ]:
feature_group_bike.create()


To persist the feature group and save feature data along with the metadata in the feature store, call the `materialise()` method with data frame.

In [ ]:
feature_group_bike.materialise(bike_df)

<a id="explore_featuregroup"></a>
### 3.3. Explore feature groups

You can retrieve feature data in a DataFrame, that can either be used directly to train models or materialized to file(s) for later use to train models

In [ ]:
query = feature_group_bike.select() 
query.show()

You can call the `get_statistics()` method of the feature group to fetch statistics for a specific ingestion job.You can use `to_pandas()` or `to_json()` to view the statistics.

In [ ]:
feature_group_bike.get_statistics().to_pandas()

You can visualize feature statistics with `to_viz()`

In [ ]:
feature_group_bike.get_statistics().to_viz()

In [ ]:
feature_group_bike.get_statistics().to_viz(["birthyear"])

You can call the `get_validation_output()` method of the FeatureGroup instance to fetch validation results for a specific ingestion job.

In [ ]:
feature_group_bike.get_validation_output().to_pandas()

In [ ]:
feature_group_bike.get_validation_output().to_summary()

<a id="visualise_lineage"></a>
#### Visualise lineage

Use the ```.show()``` method on the FeatureGroup instance to visualize the lineage of the featuregroup.

In [ ]:
feature_group_bike.show()

<a id="create_dataset"></a>
### 3.4 Create dataset
A dataset is a collection of feature snapshots that are joined together to either train a model or perform model inference.

In [ ]:
query.to_string()

In [ ]:
dataset_resource = (
    Dataset()
    .with_description("Dataset consisting of a subset of features in feature group: bike riders")
    .with_compartment_id(compartment_id)
    .with_name("bike_riders_dataset")
    .with_entity_id(entity.id)
    .with_feature_store_id(feature_store.id)
    .with_query(query.to_string())
)

In [ ]:
dataset = dataset_resource.create()

You can call the `materialise()` method of the Dataset instance to load the data to dataset.

In [ ]:
dataset.materialise()

<a id="explore_dataset"></a>
### 3.5 Explore dataset

In [ ]:
dataset.as_of(version_number=0).show()

In [ ]:
dataset.get_statistics().to_pandas()

In [ ]:
dataset.get_statistics().to_viz()

In [ ]:
dataset.profile().show()

<a id="visualise_lineage"></a>
#### Visualise lineage

Use the ```.show()``` method on the Dataset instance to visualize the lineage of the dataset.

In [ ]:
dataset.show()

<a id="featurestore_yaml"></a>
# 4. Feature store quick start using YAML
In an ADS feature store module, you can either use the Python programmatic interface or YAML to define feature store entities. Below section describes how to create feature store entities using YAML as an interface.

In [ ]:
feature_store_yaml = """
apiVersion: v1
kind: featureStore
spec:
  displayName: Bike feature store
  compartmentId: <compartment_id>
  offlineConfig:
    metastoreId: <metastore_id>

  entity: &bike_entity
    - kind: entity
      spec:
        name: Bike rides

  featureGroup:
    - kind: featureGroup
      spec:
        entity: *bike_entity
        name: bike_feature_group
        primaryKeys:
          - bikeid
        inputFeatureDetails:
          - name: "bikeid"
            featureType: "INTEGER"
            orderNumber: 1
            cast: "STRING"
          - name: "endstationlongitude"
            featureType: "FLOAT"
            orderNumber: 2
            cast: "STRING"
          - name: "tripduration"
            featureType: "INTEGER"
            orderNumber: 3
            cast: "STRING"

  dataset:
    - kind: dataset
      spec:
        name: bike_dataset
        entity: *bike_entity
        description: "Dataset for bike"
        query: 'SELECT bike.bikeid, bike.endstationlongitude FROM bike_feature_group bike'
"""

In [ ]:
registrar = FeatureStoreRegistrar.from_yaml(yaml_string=feature_store_yaml)
registrar.create()

<a id='ref'></a>
# References

- [Feature Store Documentation](https://feature-store-accelerated-data-science.readthedocs.io/en/latest/overview.html)
- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)